# Objects

# Creating objects with fields and methods

Objects bundle data and related functions together to make useful things.  If functions are reusable verbs, then objects are reusable nouns.

We've now spent some time using existing objects, such as the Pandas DataFrame.  Now we'll see how to define our own.


Let's consider a restaurant bill as a candidate for becoming an object, since it has several pieces of information associated with it and some different things you can do with it.  A Bill contains a few pieces of information:  a list of items purchased and their prices, a total cost before and after tax, a tip, and whether it was paid.  We have a choice about how to represent some information, like the total price before and after tax, because it might either be stored or computed on the fly.


To create an object type, we need to declare a class.  This is like the template that can create multiple object instances according to the same instructions.

In [ ]:
class Bill:
  """ Represents a bill at a restaurant.

  items (list of tuples):  list of (item name, cost) tuples
  """
  def __init__(self, items):
    self._items = items

  # Just a "getter" method - reasons for this explained later
  def items(self):
    return self._items

  # "Setter" method
  def set_items(self, items):
    self._items = items
  
  def total_cost_pretax(self):
    total = 0
    for name, cost in self._items:
      total += cost
    return total

  def total_cost_with_tax(self, tax_rate):
    return self.total_cost_pretax() * (1 + tax_rate)


In [ ]:
my_lunch = [("Ham Sandwich", 9), ("Coke", 2)]
new_bill = Bill(my_lunch) # Calls the __init__() constructor
print(f"Total cost: {new_bill.total_cost_with_tax(0.08):.2f}")

```__init__()``` is a constructor, and has a special name so that the interpreter knows it's a constructor (the constructor always has exactly that name).  It's called by naming the class, then adding any necessary arguments between the parentheses.  Calling the constructor creates an object of the Class's type - an *instance*.

```_items``` is an instance attribute, also known as a field.  Each copy of the object that we create will have its own ```_items``` attribute, set in the constructor to hold a list of tuples.  While many instance attributes can be accessed from anywhere with the syntax name_of_object.attribute, the underscore at the beginning of this name says we're supposed to leave it alone outside the class.

Each method takes "self" as its first argument, but we don't actually pass in anything for this when we call the method.  That is just a way to refer to the object itself in the method.  Leaving "self" out is legal but makes the function "static" (see later).

It's a little unusual compared to other languages like Java, but we don't need to explicitly name what fields of the object exist.  We can later assign to a new variable name that we haven't mentioned before, and the object will just behave as if that field always existed.  (Even if the new name was a typo, sadly.)

In [ ]:
new_bill.foo = 5
print(new_bill.foo)

In the body of the constructor, we assign the arguments of the function to the fields of the object, which are always preceded by "self." in the method code.  Outside the class code, we can refer to fields using the syntax objectname.fieldname, as demonstrated above.  So we could just not use the methods we created for accessing the data, and get it directly.

However, it's generally considered a better practice to work with the "setter" and "getter" methods for an object, the methods designed to replace accessing fields directly.  One reason for this practice is to allow easy changes to the underlying representation of the object, without needing to change code that interacts with it.  If we shouldn't touch an attribute outside the class code, it should start its name with an underscore.


Here's an example of why code tends to interact with attributes/fields through methods instead of directly.  We'll see two ways of representing a fit to data in a moment.

The first just needs to be told what the fit is and remembers it.

The second actually computes the fit from the points (don't worry about that code) but doesn't do so until its fit() method is called.  Trying to access fields in the second version before fit() is called would result in a crash with Python claiming the fields don't exist.

In [ ]:
class LinearFit:
  def __init__(self, x, y, m, b):
    self.x = x # list of x-coordinates
    self.y = y # list of y-coordinates
    self.m = m
    self.b = b

  def points(self):
    return self.x, self.y

  def fit(self):
    return self.m, self.b

In [ ]:
linearfit = LinearFit([1,2,3],[1,2,3],1,0)
print(linearfit.points())

In [ ]:
import numpy as np

class LinearFit2:
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.computed_fit = False

  def points(self):
    return self.x, self.y

  def fit(self):
    if (self.computed_fit):
      return self._m, self._b
    # Compute the fit
    A = np.transpose(np.concatenate (([self.x], np.ones((1,len(self.x))))))
    fit = np.linalg.lstsq(A,self.y,rcond=None)
    self._m = fit[0][0]
    self._b = fit[0][1]
    self.computed_fit = True
    return self._m, self._b

In [ ]:
linearfit = LinearFit2([1,2,3],[1,2,3])
print(linearfit.fit())

In the second version, the code is "lazy" and doesn't compute the fit until it is needed.  If we were to try to examine the fields without calling fit(), we'd get an error.  


As a second example, a circle's size is defined by one parameter.  But should that parameter be the radius, the diameter, the area, or the circumference?  Code that interacts with the methods to get and set values need not worry about what happens when the underlying representation changes.

In [ ]:
import math

class Circle:
  def __init__(self, radius):
    self._radius = radius
  
  def radius(self):
    return self._radius

  def area(self):
    return math.pi * self._radius ** 2
  
  def diameter(self):
    return 2 * self._radius
  
  def circumference(self):
    return 2 * math.pi * self._radius

my_circle = Circle(2)
print(my_circle.area())

In [ ]:
import math

class Circle2:
  def __init__(self, radius):
    self._diameter = radius*2
  
  def radius(self):
    return self._diameter/2

  def area(self):
    return math.pi * self.radius() ** 2
  
  def diameter(self):
    return self._diameter
  
  def circumference(self):
    return math.pi * self._diameter

my_circle = Circle(2)
print(my_circle.area())

# Validation in the Constructor

One use of objects is in validating data - making sure it's the right type and range for its intended use.  This also demonstrates the use of *raise*, or raising your own exception.

In [ ]:
class Circle3:
  def __init__(self, radius):
    if radius < 0:
      raise ValueError("Can't have negative circle radius")
    self.radius=radius

Circle3(-1)

# Default parameter values

It often makes sense to have the constructor take all the attributes of an object as arguments.  But there may be some default values that work well.
You can set default arguments to the constructor by following each argument with =value, where value is the default value.

In [ ]:
class Circle4:
  def __init__(self,radius=2):
    self.radius = radius

Circle4().radius

# Choosing what is an object

Suppose we're writing software for a gradebook.  There are students who have grades on assignments and tests.  It's for a particular class.  What among these might usefully be an object?



The students themselves are good candidates for being objects, because each has a wealth of data associated with them (multiple assignments and tests) and verb-like actions we could do to them (add or remove).

The Gradebook itself might be an object, even though there's only one of it, because it also has a lot of data associated with it (many students) and actions we could perform on it (sort, find averages).


The grades themselves are probably too small and simple to need to be objects, although we could just for the validation aspect (ensuring grades when created are between 0 and 100, for example).

The particular course name, such as "Geology 101," probably is best as an attribute of the gradebook instead of its own class.

In short, things that make good classes often have several pieces of data associated with them, along with functions that specifically pertain to that data.  But choosing what to treat as an object is often a question of elegance rather than a hard-and-fast rule.

# Exercise

Try defining an object that represents a student.  Include at least 3 variables that describe the student, including age.  Define a constructor and a method get_older() which increases the student's age by the argument.  You don't need to define all the getters and setters.

# Pass by reference and objects

Like lists and dictionaries, objects are considered heavyweight enough that an assignment doesn't copy the underlying data - it just hands off a reference to the same data.  We can actually see this because the default way to print an object prints its address.
(Addresses are numbers in hexadecimal, base 16, where a = 10, b = 11, etc, f = 15.)

In [ ]:
lf = LinearFit([1],[2],3,4)
lf2 = lf
lf2.x = [10]
print(lf.x)  # Will say 10 - we modified the underlying array, affecting both
print(lf)
print(lf2)

You can see from the print statements that these are both talking about the same object instance.

# Privacy conventions

Naming an attribute with a leading underscore is a hint that it's not meant to be accessed directly outside the class.  Naming an attribute with two leading underscores says it's *really* not meant to be accessed except by the class, and Python will "mangle" the name of the variable by adding _ClassName to it just to help ensure it's not accessed by accident.  But in all cases, unlike in other languages (C++,Java), there's no way to absolutely prevent attributes from being accessed.

# Static methods

The above methods show how you can call methods from objects, and use the objects' information.  But sometimes, you want a kind of "mothership" for the objects that has general functionality associated with the objects, without being one of them.  That's a class, and the "static" methods associated with a class don't refer to the "self," because they're called without having an instance available.

In short, the class can have its own fields and methods which don't require an object to be created.  The classname, instead of an object's name, is used to access these.  The fields and methods are called "static."

One use for this might be to track how many objects we've created.  The count of zero can't be stored in an object when there are no objects yet.

In [ ]:
class Robot:
  robot_count = 0 # class attribute

  def __init__(self):
    self.name = "robot" + str(Robot.robot_count)
    print("Hello, " + self.name + "!")
    Robot.update_robot_count() # static method, uses Class name

  def update_robot_count(): # notice no "self"
    Robot.robot_count += 1

In [ ]:
Robot1 = Robot()
Robot2 = Robot()
Robot3 = Robot()

In [ ]:
Robot.robot_count

It's very easy to accidentally make static methods, by leaving "self" out of the arguments.

# Exercise

Modify the robot code above to have a non-static method pew(), a static robots_destroyed variable, and a non-static dead variable.  On calling robot.pew(target), if the target or the shooter (self) was dead, do nothing.  If both are alive, announce that the caller of pew() is firing at its target, set the target to dead, increment robots_destroyed, and print the value of robots_destroyed.

In [ ]:
class Robot:
  robot_count = 0 # class attribute
  # TODO robots_destroyed initialized here

  def __init__(self):
    self.name = "robot" + str(Robot.robot_count)
    print("Hello, " + self.name + "!")
    Robot.update_robot_count() # static method, uses Class name

  def update_robot_count(): # notice no "self"
    Robot.robot_count += 1
    
  def pew(self, target):
    # Fire at and destroy another robot.
    # Check that both are alive TODO
      print(f'{self.name} fires at {target.name}!')
      # TODO make target dead
      # TODO update Robot.robots_destroyed
      print(f'Robots destroyed: {Robot.robots_destroyed}')